In [1]:
import json
import chromadb

# 1. SETUP: Initialize ChromaDB
# Using PersistentClient ensures data is saved to disk (in a folder named 'chroma_db')
# If you just want in-memory for testing, use chromadb.Client()
client = chromadb.PersistentClient(path="./transition_db")

# Create (or get) the collection
# By default, this uses the 'all-MiniLM-L6-v2' embedding model (great for semantic similarity)
collection = client.get_or_create_collection(name="premiere_transitions")

# 2. DATA PREPARATION: Load and Format
# Load the JSON file you generated in the previous step
with open('transitionlist.json', 'r') as f:
    transition_data = json.load(f)

ids = []
documents = []
metadatas = []

# Iterate through the dictionary items to "divide exactly"
for name, description in transition_data.items():
    # ID: unique identifier (The transition name serves this purpose perfectly)
    ids.append(name)
    
    # DOCUMENT: This is what the AI "reads" and embeds. 
    # We combine Name + Description to ensure the vector captures the full context.
    text_content = f"{name}: {description}"
    documents.append(text_content)
    
    # METADATA: Structured data to return to your app (The clean name)
    metadatas.append({"transition_name": name, "raw_description": description})

# 3. INGESTION: Add to ChromaDB
# Chroma handles the tokenization and embedding automatically here
print(f"Adding {len(ids)} transitions to the database...")
collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)
print("Ingestion complete.")

# --- SIMULATION OF YOUR USE CASE ---

# 4. RAG QUERY: Searching based on "Vibe"
# This simulates the LLM's output describing a needed vibe
vibe_query = "I want something that looks like a glitchy computer error or broken screen"

results = collection.query(
    query_texts=[vibe_query],
    n_results=3 # Return top 3 matches
)

# 5. PARSE RESULTS
print(f"\nQuery: '{vibe_query}'\n")
print("Top Matches:")

# Chroma returns lists of lists (batch processing), so we access index [0]
for i in range(len(results['ids'][0])):
    match_id = results['ids'][0][i]
    match_meta = results['metadatas'][0][i]
    match_dist = results['distances'][0][i] # Lower distance = better match
    
    print(f"Name: {match_meta['transition_name']}")
    print(f"Desc: {match_meta['raw_description']}")
    print(f"Distance: {match_dist:.4f}")
    print("-" * 30)

Adding 115 transitions to the database...
Ingestion complete.

Query: 'I want something that looks like a glitchy computer error or broken screen'

Top Matches:
Name: AE.AE_Impact_Glitch
Desc: Simulates digital data corruption, pixel sorting, and compression errors for a sci-fi or tech-fail vibe.
Distance: 1.2073
------------------------------
Name: AE.AE_Impact_Flicker
Desc: Rapidly toggles brightness or opacity, simulating a strobing light or broken projector.
Distance: 1.2761
------------------------------
Name: ADBE Wipe
Desc: A simple, hard-edged line travels across the screen to replace one clip with another.
Distance: 1.3058
------------------------------


In [ ]:
import json
from faster_whisper import WhisperModel
from better_profanity import profanity

# 1. Setup Models (Load once on server start)
# 'tiny' or 'base' is usually enough for profanity and very fast.
model = WhisperModel("base", device="cpu", compute_type="int8")
profanity.load_censor_words() 

# You can add custom words relevant to your users
custom_bad_words = ["specific_word", "another_bad_word"]
profanity.add_censor_words(custom_bad_words)

def detect_cursed_words(audio_path):
    print(f"Analyzing: {audio_path}")

    # 2. Transcribe with Word Timestamps
    # 'word_timestamps=True' is the magic key here
    segments, info = model.transcribe(audio_path, word_timestamps=True)

    markers = []

    # 3. Iterate linearly through the timeline
    for segment in segments:
        for word in segment.words:
            # Clean the word for checking (remove punctuation)
            clean_word = word.word.strip(".,!?\"'")
            
            # 4. Check Profanity
            if profanity.contains_profanity(clean_word):
                
                # Calculate duration
                duration = word.end - word.start
                
                # Create the Marker Object
                markers.append({
                    "start_seconds": round(word.start, 3),
                    "duration_seconds": round(duration, 3),
                    "name": "PROFANITY",
                    "comment": f"Detected word: '{clean_word}' (Confidence: {int(word.probability * 100)}%)",
                    "color_index": 0 
                })

    # 5. Return JSON to UXP
    return {"markers": markers}

# --- Mock Output ---
# result = detect_cursed_words("C:/path/to/audio.wav")
# print(json.dumps(result, indent=2))


# SEVERITY_MAP = {
#     "damn": "Mild",
#     "hell": "Mild",
#     # ... others
# }

# # Inside the loop:
# severity = SEVERITY_MAP.get(clean_word.lower(), "Critical")
# color_index = 0 if severity == "Mild" else 1 # Different colors for severity